In [1]:
# assign directory
import os
BRANDON = '/Users/brandonmarks/Desktop/Research Materials/hierarchical-bayesian-model-validation/'
YASH = '/Users/yashd/Desktop/hierarchical-bayesian-model-validation/'

ROOT_DIR = BRANDON
data_dir = ROOT_DIR + 'raw-data/100_syntheticMRI-3d'
# iterate over files in that directory
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)

In [2]:
os.chdir(ROOT_DIR + "dataset-preparation/")
from transform_utilities import *

# Fourier

In [3]:
first_image = nib.load(file_list[0]).get_fdata()
mri_coord_df = getIndexDF_3d(first_image).sort_values(["magnitude"])

In [4]:
#Values obtained from plots in axial_fourier.ipynb
splits = getSplits(0.14717525215517308, 0.7491928054205256, 1.05)
print(splits)

[0.14717525215517308, 0.15453401476293174, 0.16226071550107835, 0.17037375127613227, 0.17889243883993888, 0.18783706078193582, 0.1972289138210326, 0.20709035951208424, 0.21744487748768845, 0.22831712136207288, 0.23973297743017655, 0.2517196263016854, 0.26430560761676963, 0.2775208879976081, 0.29139693239748854, 0.30596677901736297, 0.32126511796823115, 0.3373283738666427, 0.3541947925599749, 0.37190453218797365, 0.39049975879737236, 0.410024746737241, 0.43052598407410303, 0.4520522832778082, 0.47465489744169864, 0.4983876423137836, 0.5233070244294727, 0.5494723756509464, 0.5769459944334938, 0.6057932941551686, 0.636082958862927, 0.6678871068060733, 0.701281462146377, 0.7363455352536958]


In [5]:
gray_fr = convert_to_fourier_basis_3d(data_dir, split_list = splits, coord_df=mri_coord_df, debug= True)
#pd.to_pickle(gray_fr, '../transformed-data/toy-agriVision-red-fourier.pickle')
gray_fr.head()

100%|██████████| 100/100 [00:35<00:00,  2.85it/s]


[0.14717525 0.15453401 0.16226072 0.17037375 0.17889244 0.18783706
 0.19722891 0.20709036 0.21744488 0.22831712 0.23973298 0.25171963
 0.26430561 0.27752089 0.29139693 0.30596678 0.32126512 0.33732837
 0.35419479 0.37190453 0.39049976 0.41002475 0.43052598 0.45205228
 0.4746549  0.49838764 0.52330702 0.54947238 0.57694599 0.60579329
 0.63608296 0.66788711 0.70128146 0.73634554]


100%|██████████| 34/34 [00:11<00:00,  2.98it/s]


,band,magnitude_endpoints,unique_magnitudes,data
0,1,"(0.0, 0.1471698354255507)",4577,"[49257324416.0, 49257324416.0, 49257324416.0, ..."
1,2,"(0.14717525215517308, 0.15452878504719844)",621,"[-27141263.44444302, -27141263.44444302, -2714..."
2,3,"(0.1545365231581063, 0.16225943732183962)",707,"[-50701152.42484738, -50701152.42484738, -5070..."
3,4,"(0.16226926318779597, 0.17037202315964378)",800,"[-44436270.390244916, -44436270.390244916, -44..."
4,5,"(0.17038839934882027, 0.178888112049552)",938,"[12953747.345902056, 12953747.345902056, 12953..."


In [6]:
gray_fr = fourier_full_decomp_3d(data_dir, coord_df= mri_coord_df, debug = True)
gray_fr.head()

100%|██████████| 100/100 [00:36<00:00,  2.77it/s]


,x_index,y_index,z_index,x_freq,y_freq,z_freq,magnitude,Data
index_coords,,,,,,,,
"(0, 0, 0)",0,0,0,0.00000,0.000000,0.00000,0.000000,"[(49257324416-0j), (49257324416-0j), (49257324..."
"(0, 1, 0)",0,1,0,0.00000,0.004464,0.00000,0.004464,"[(-18493530034.481766-3943535717.4455204j), (-..."
"(0, 0, 1)",0,0,1,0.00000,0.000000,0.00625,0.006250,"[(-10388715531.239899-9243585541.701508j), (-1..."
"(1, 0, 0)",1,0,0,0.00625,0.000000,0.00000,0.006250,"[(-13007545690.156769+810555099.807267j), (-13..."
"(1, 1, 0)",1,1,0,0.00625,0.004464,0.00000,0.007681,"[(1550923540.2460556+1526898431.9516926j), (15..."


# Wavelet

In [7]:
def filter_func(x):
    x_copy = x.copy().astype(float)
    filtered = ndimage.uniform_filter(x, size = 3)
    #filtered = ndimage.median_filter(x.astype("float"), size = 7)
    x_copy[np.isclose(filtered, 0)] = np.nan
    #x_copy[filtered <= 0.05] = np.nan
    return x_copy
gray_wv = convert_to_wavelet_basis_3d(data_dir, image_func= filter_func, debug = True)
gray_wv["data"] = gray_wv["data"].apply(lambda x: x[~np.isnan(x)])
gray_wv.head(10)

8 layers being used


100%|██████████| 100/100 [00:38<00:00,  2.61it/s]


,layer,orientation,data
0,1,L1,[]
1,2,aad,[]
2,2,ada,[]
3,2,add,[]
4,2,daa,[]
5,2,dad,[]
6,2,dda,[]
7,2,ddd,[]
8,3,aad,"[-1178617.707031251, -180064.888671875, -10603..."
9,3,ada,"[-163132.13281250038, -128205.81445312491, -28..."
